In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/sergeywong/cp-vton

In [ ]:
cd /content/cp-vton

In [ ]:
!pip -q install tensorboardX

In [ ]:
!python data_download.py

Subset 1000 image pairs for training


In [ ]:
with open('/content/cp-vton/data/train_pairs.txt') as f:
  pairs = f.readlines()
  pairs = [pair.replace('\n', '') for pair in pairs]
  pairs = pairs[:1000]
  sub_cloth = []
  sub_image = []
  for pair in pairs:
    sub_cloth.append(pair.split(' ')[1])
    sub_image.append(pair.split(' ')[0])

sub_parse = [file.replace('jpg', 'png') for file in sub_image]
sub_pose = [file.replace('.jpg', '_keypoints.json') for file in sub_image]

In [ ]:
sub_pose[:10]

In [ ]:
!mv /content/cp-vton/data/train/cloth /content/cp-vton/data/train/cloth1
!mv /content/cp-vton/data/train/cloth-mask /content/cp-vton/data/train/cloth-mask1
!mv /content/cp-vton/data/train/image /content/cp-vton/data/train/image1
!mv /content/cp-vton/data/train/image-parse /content/cp-vton/data/train/image-parse1
!mv /content/cp-vton/data/train/pose /content/cp-vton/data/train/pose1
!mv /content/cp-vton/data/train_pairs.txt /content/cp-vton/data/train_pairs1.txt

In [ ]:
!mkdir /content/cp-vton/data/train/cloth
!mkdir /content/cp-vton/data/train/cloth-mask
!mkdir /content/cp-vton/data/train/image
!mkdir /content/cp-vton/data/train/image-parse
!mkdir /content/cp-vton/data/train/pose

In [ ]:
import glob
import shutil, os

folder_cloth = 'cloth'
folder_mask = 'cloth-mask'
folder_image = 'image'
folder_parse = 'image-parse'
folder_pose = 'pose'

def copy_files(folder, sub_, mode):
    for file in sub_:
      src = r'/content/cp-vton/data/' + mode + '/' + folder +'1/' + file
      dst = r'/content/cp-vton/data/' + mode +'/' + folder + '/' + file
      shutil.copyfile(src, dst)

In [ ]:
# sub-cloth, image, parse, pose
copy_files(folder_parse, sub_parse, 'train')
copy_files(folder_pose, sub_pose, 'train')

In [ ]:
with open('/content/cp-vton/data/train_pairs.txt', 'w') as f:
  for item in pairs:
    f.write('%s\n' % item)

Subset 1000 image pairs for testing

In [ ]:
with open('/content/cp-vton/data/test_pairs.txt') as f:
  pairs = f.readlines()
  pairs = [pair.replace('\n', '') for pair in pairs]
  pairs_t = pairs[:1000]
  sub_cloth_t = []
  sub_image_t = []
  for pair in pairs_t:
    sub_cloth_t.append(pair.split(' ')[1])
    sub_image_t.append(pair.split(' ')[0])

sub_parse_t = [file.replace('jpg', 'png') for file in sub_image_t]
sub_pose_t = [file.replace('.jpg', '_keypoints.json') for file in sub_image_t]

In [ ]:
!mv /content/cp-vton/data/test/cloth /content/cp-vton/data/test/cloth1
!mv /content/cp-vton/data/test/cloth-mask /content/cp-vton/data/test/cloth-mask1
!mv /content/cp-vton/data/test/image /content/cp-vton/data/test/image1
!mv /content/cp-vton/data/test/image-parse /content/cp-vton/data/test/image-parse1
!mv /content/cp-vton/data/test/pose /content/cp-vton/data/test/pose1
!mv /content/cp-vton/data/test_pairs.txt /content/cp-vton/data/test_pairs1.txt

In [ ]:
!mkdir /content/cp-vton/data/test/cloth
!mkdir /content/cp-vton/data/test/cloth-mask
!mkdir /content/cp-vton/data/test/image
!mkdir /content/cp-vton/data/test/image-parse
!mkdir /content/cp-vton/data/test/pose

In [ ]:
# sub-cloth, image, parse, pose
copy_files(folder_cloth, sub_cloth_t, 'test')
copy_files(folder_mask, sub_cloth_t, 'test')
copy_files(folder_image, sub_image_t, 'test')
copy_files(folder_parse, sub_parse_t, 'test')
copy_files(folder_pose, sub_pose_t, 'test')

Prepare dir to load from checkpoint

In [ ]:
!mkdir /content/cp-vton/checkpoints

Train GMM

In [ ]:
# in cp_dataset.py: https://github.com/sergeywong/cp-vton/issues/26

In [ ]:
# in networks.py: https://github.com/sergeywong/cp-vton/issues/30

In [ ]:
!python train.py --name gmm_train_new --stage GMM --workers 4 --save_count 5000 --display_count 100 --shuffle --checkpoint checkpoints/gmm_step_100000.pth

Check for checkpoint

In [ ]:
cd /content/cp-vton/checkpoints/gmm_train_new/

In [ ]:
!ls -l

In [ ]:
from google.colab import files
files.download('/content/cp-vton/checkpoints/gmm_train_new/step_100000.pth') 

In [ ]:
cd /content/cp-vton

Check Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/cp-vton/tensorboard/gmm_train_new

Generate GMM clothes for TOM

In [ ]:
!python test.py --name gmm_traintest_new --stage GMM --workers 4 --datamode train --data_list train_pairs.txt --checkpoint /content/drive/MyDrive/gmm_step_200000.pth

In [ ]:
!zip -r "/content/cp-vton/result/gmm_step_200000.pth/train.zip" "/content/cp-vton/result/gmm_step_200000.pth/train"

In [ ]:
cp /content/cp-vton/result/gmm_step_200000.pth/train.zip /content/drive/MyDrive

In [ ]:
cp -r /content/cp-vton/result/gmm_step_200000.pth/train/warp-cloth /content/cp-vton/data/train

In [ ]:
cp -r /content/cp-vton/result/gmm_step_200000.pth/train/warp-mask /content/cp-vton/data/train

In [ ]:
!rm -rf /content/cp-vton/result/gmm_step_200000.pth/train

Import warping for TOM train

In [ ]:
cp /content/drive/MyDrive/warp.zip /content/cp-vton/data

In [ ]:
!unzip /content/cp-vton/data/warp.zip -d /content/cp-vton/data

Train TOM

In [ ]:
!python train.py --name tom_train_new --stage TOM --workers 4 --save_count 5000 --display_count 500 --batch-size 8 --shuffle --checkpoint /content/cp-vton/checkpoints/tom_train_new/step_010000.pth

Check Tensorboard

In [ ]:
%tensorboard --logdir /content/cp-vton/tensorboard/tom_train_new

Download checkpoint

In [ ]:
cd /content/cp-vton/checkpoints/tom_train_new/

In [ ]:
!ls -l

In [ ]:
files.download('/content/cp-vton/checkpoints/tom_train_new/step_050000.pth') 

In [ ]:
cd /content/cp-vton

TOM evaluation

In [ ]:
!python test.py --name tom_test_new --stage TOM --workers 4 --datamode train --data_list train_pairs.txt --checkpoint /content/drive/MyDrive/tom_step_060000.pth

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/cp-vton/tensorboard/tom_test_new